# **A JUMBLED introduction to Graph Theory**

# [Köningsberg bridges](https://www.wikiwand.com/en/Seven_Bridges_of_K%C3%B6nigsberg)

![""](Koningsberg-bridges.png)

**Euler** was asked in 1736 to find a nice path across the seven Köningsberg bridges that visits the 

![](Koningsberg-graph.png)

# What we are going to talk about:
- Traversal
- Connected Components
- Depth First Search
- Topological Sorting
- Breadth First Search (?)
- Minimum Spanning Tree (?)

# Definition
A graph $G = (V, E)$ is 
- a pair of vertices (or nodes) $V$, and
- a set of edges $E$, 
- both assumed finite i.e. 
    - $|V| = n$ and $|E| = m$.

### Example of a simple graph:

![](sparse-graph.png)

$\LARGE V = \{1, 2, 3, 4, 5, 6\}$

$\LARGE E = \{(1,4), (2,4), (3,4), (4,5), (5,6)\}$

$\LARGE n = 6, m = 5$




### Example of a multi graph:
(a graph with multi-edges (AKA parallel edges), or edges that have the same endpoints)
![Sample Directed Graph](sample-graph.png)

# Graph Representation
* adjacency matrix
* adjacency list
* adjacency set
* adjacency dictionary
* left-child right sibling

We are sticking to **adjacency sets**. Their benefits: 
* as compact as adjacency lists for **sparse** graphs
* O(1) look up, because sets!

In [1]:
G = {
'a': set('bcdef'),
'b': set('ce'),
'c': set('d'),
'd': set('e'),
'e': set('f'),
'f': set('cgh'),
'g': set('fh'),
'h': set('fg')
}

## Density of a graph:

\begin{equation*}
\LARGE D = \frac{2|E|}{|V|\,(|V|-1)}
\end{equation*}


## pop quiz:
n people are clinking their drinks. Assuming no 2 clinks happen at the same time, how many of them will you hear?

### Sparse graph:
\begin{equation*}
\LARGE|E| = O(|V|)
\end{equation*}

![](sparse-graph.png)

### Dense graph (this graph is also a complete graph):
\begin{equation*}
\LARGE |E| = O(|V|^2)
\end{equation*}
![](complete-graph.png)

In [33]:
G = {
'a': set('bcdef'),
'b': set('ce'),
'c': set('d'),
'd': set('e'),
'e': set('f'),
'f': set('cgh'),
'g': set('fh'),
'h': set('fg')
}

In [3]:
G

{'a': {'b', 'c', 'd', 'e', 'f'},
 'b': {'c', 'e'},
 'c': {'d'},
 'd': {'e'},
 'e': {'f'},
 'f': {'c', 'g', 'h'},
 'g': {'f', 'h'},
 'h': {'f', 'g'}}

## Is node _`a`_ connected to node _b_?

In [4]:
'b' in G['a']  # O(1)

True

## Degree of node $a$

In [5]:
len(G['a']) # degree of node a (how many nodes are connected to node a)

5

## Adjacency Matrix 

In [6]:
a, b, c, d, e, f, g, h = range(8)
# a b c d e f g h
G2 = [
    [0,1,1,1,1,1,0,0], # a
    [0,0,1,0,1,0,0,0], # b
    [0,0,0,1,0,0,0,0], # c
    [0,0,0,0,1,0,0,0], # d
    [0,0,0,0,0,1,0,0], # e
    [0,0,1,0,0,0,1,1], # f
    [0,0,0,0,0,1,0,1], # g
    [0,0,0,0,0,1,1,0]  # h 
] 

In [7]:
G2[a][b]  # is b connected to a

1

In [8]:
sum(G2[a]) # degree of node a (how many nodes are connected to node a)

5

# Graph Theory Applications

- crawlers in search engines
- task schedulers
- map traversal (Google map, etc)
- Image processing (segmentation, etc)
- Assignment problems
    - How to assign workers to jobs
    - How to assign machines to locations
- Social media
- etc

## Traversal:
![](walk.png)

In [34]:
def walk(G, s, S=set()): # Walk the graph from node s
    P, Q = dict(), set() # Predecessors + "to do" queue
    P[s] = None # s has no predecessor
    Q.add(s) # We plan on starting with s
    while Q: # Still nodes to visit
        u = Q.pop() # Pick one, arbitrarily
        for v in G[u].difference(P, S): # New nodes?
            Q.add(v) # We plan to visit them!
            P[v] = u # Remember where we came from
    return P # The traversal 

In [35]:
traversal_path = walk(G,'a') # starting from node a
print("result: \n", traversal_path)

result: 
 {'a': None, 'e': 'a', 'd': 'a', 'c': 'a', 'b': 'a', 'f': 'a', 'h': 'f', 'g': 'f'}


## Traversal Path:
Shown in red

![](traversal-tree.png)

## Notes:
- nodes are arbitrarily selected from the queue
- when adding new nodes, only add the ones that are not already visited
- each time we add a new node to the queue, we set its predecessor; that is, we make sure we remember where we came from when we found it. That way we can rebuild the **traversal tree**.
- `G[u]` is a set. but the `.difference()` method works with any iterable.
- what happens if you start the walk from node 'b'?


## Question: 
Is there a nice path across the seven Köningsberg bridges that crosses each bridge only once and returns to the starting point? (we are allowed to visit each island more than once)
#### - Answer: 
No
#### - Why?
**Euler** proved that a necessary condition for the existence of such path is that all vertices in the graph have an **even degree** (Why?)

#### What is an Eulerian Cycle?

![](euler.gif)

See Also: [walk & path](https://www.wikiwand.com/en/Path_(graph_theory)), [cycle](https://www.wikiwand.com/en/Cycle_(graph_theory)), [hamiltonian path](https://www.wikiwand.com/en/Hamiltonian_path)

## Connected Components:
- in an **undirected graph**
- defined as the collection of nodes that are pair-wise accessible to each other through a **path**

![](connected-components.png)

In [36]:
G = {
'a': set('bcd'),
'b': set('ad'),
'c': set('ad'),
'd': set('abc'),
'e': set('fg'),
'f': set('eg'),
'g': set('ef'),
'h': set('i'),
'i': set('h')
}

## Questions:
#### 1. will we visit all the nodes all the time if we use the above `walk()` function?
#### 2. How can we detect all the connected components in a graph?

## Answers:
1. No. Depending on the node we start the walk from, some nodes may not be reachable.
2. We make sure to iterate over all the nodes and re-start the walk on the nodes that are not reached yet in previous walks. Function `components()` below.

In [37]:
# %load components.py
def components(G): # The connected components
    comp = []
    seen = set() # Nodes we've already seen
    for u in G: # Try every starting point
        if u in seen: continue # Seen? Ignore it
        C = walk(G, u) # Traverse component
        seen.update(C) # Add keys of C to seen
        comp.append(C) # Collect the components
    return comp


comps = components(G)
comps

[{'a': None, 'b': 'a', 'c': 'a', 'd': 'a'},
 {'e': None, 'g': 'e', 'f': 'e'},
 {'h': None, 'i': 'h'}]

## Strongly connected comopnents:
- in a **directed graph**
- defined as the collection of nodes that are pair-wise accessible to each other through a **path**

![](strongly-connected-components.png)


**Methods** to solve Strongly Connected Components? (we will come back to this after **topological sorting**)
- Kosaraju's algorithm 
- Tarjan's algorithm

# Depth-first-search traversal


- you are wearing muddy boots and walking in a maze.
- you **backtrack** any time you reach a dead end.
- DFS from a single node will traverse all the nodes that are connected to the starting node (not necessarily the whole graph)
- [example](https://visualgo.net/en/dfsbfs) (raise your hand if the example is not clear to you)

In [38]:
def iter_dfs(G, s):
    S, Q = set(), [] # Visited-set and queue
    Q.append(s) # We plan on visiting s
    while Q: # Planned nodes left?
        u = Q.pop() # Get one
        if u in S: continue # Already visited? Skip it
        S.add(u) # We've visited it now
        Q.extend(G[u]) # Schedule all neighbors
        yield u # Report u as visited

In [39]:
for node in iter_dfs(G, 'a'):
    print(node)

a
d
c
b


In [40]:
for node in iter_dfs(G, 'b'):
    print(node)

b
d
a
c


In [41]:
for node in iter_dfs(G, 'e'):
    print(node)

e
f
g


In [42]:
for node in iter_dfs(G, 'h'):
    print(node)

h
i


# topological sort

Input: A Directed Acyclic Graph

Ouptut: an ordering of the nodes in such a way, that if there is an edge directed towards vertex $v_j$ from vertex $v_i$, then $v_i$ comes before $v_j$. 

![](topsortexample.png)


There are multiple topological sorting possible for a graph.

Example: $\Large [1, 2, 3, 4, 5]$ and $\Large [1, 2, 3, 5, 4]$ for the graph below



## Question 1: 

Is $\Large [1, 2, 4, 3, 5]$ a topological sorting of the graph?

## Question 2:

When does a graph have a topological sorting?

### Answer 2:

When it is a DAG. (no cycles)

Before we can continue, we need to study an extension of DFS:
# Depth First Timestamp

The thing that is particular to DFS trees is that all descendants of a node $u$ are processed in the time interval from when $u$ is discovered to when we backtrack through it.

- We keep a counter that we increment any time we discover a node or finish with a node.

In [18]:
DAG = {
'1': set('23'),
'2': set('34'),
'3': set('45'),
'4': set(),
'5': set(),
}

In [19]:


def dfs(G, s, d, f, S=None, t=0):
    
    """Traverse a graph in DFS fasion.
    Keyword arguments:
    G -- Input Graph
    s -- the starting node (sink)
    d -- Discover time (when node added to Q)
    f -- Finish time (when all the childred of a node are finished)
    S -- Forbidden Zone (List of nodes that should not be traversed)
    t -- timestamp
    """
    
    if S is None: S = set() # Initialize the history
    d[s] = t; t += 1 # Set discover time
    S.add(s) # We've visited s
    for u in G[s]: # Explore neighbors
        if u in S: continue # Already visited. Skip
        t = dfs(G, u, d, f, S, t) # Recurse; update timestamp
    f[s] = t; t += 1 # Set finish time
    return t # Return timestamp

In [20]:
import operator 

d, f = {}, {}
starting_node = '1'  # change this to '2', '3', '4' and compare the results
dfs(DAG, starting_node, d, f)

# converting to list and sorting descendingly based on finishing time
sorted_f = sorted(f.items(), key=operator.itemgetter(1), reverse=True)
print("====================")
print("Discovered times: ", d)
print("Finished times: ", f)
print("====================")
print("\nA TOPOLOGICAL sorting: ", [k[0] for k in sorted_f])
print("====================")

Discovered times:  {'1': 0, '2': 1, '4': 2, '3': 4, '5': 5}
Finished times:  {'4': 3, '5': 6, '3': 7, '2': 8, '1': 9}

A TOPOLOGICAL sorting:  ['1', '2', '3', '5', '4']


## Notes:
    - At every timestamp, we are either discovering or getting 
    finished with a single node
    - Every node is discovered before its descendants.
    - Every node is finished after its descendants.
    - The dfs function above is not iterating over all the nodes. We need to do that to 
    detect all the finishing times for all the nodes and to ensure we cover
    the entire graph

In [21]:
# we are using a trick to improve the time complexity.
# As nodes are finished, add them to a list
# eventually instead of sorting the nodes based on their 
# finish time, we can just reverse this list (cheaper than sorting)

# We are using python closures here

def dfs_topsort(G):
    S, res = set(), [] # History and result
    
    def recurse(u): # Traversal subroutine
        if u in S: return # Ignore visited nodes
        S.add(u) # Otherwise: Add to history
        for v in G[u]:
            recurse(v) # Recurse through neighbors
        res.append(u) # Finished with u: Append it
        
    for u in G:
        recurse(u) # Cover entire graph
        res.reverse() # It's all backward so far
    
    return res


dfs_topsort(G)

['e', 'g', 'f', 'a', 'b', 'd', 'c', 'i', 'h']

## Strongly Connected Components -- **Revisited**

input graph:

![](strongly-connected-components.png)


create the SCC graph by converting strongly connected components into super nodes:

![](scc.png)


## Questions:
1. Why is the SCC graph acyclic?

2. Establishing that SCC graph is acyclic, how does the finish time of any node in $A$ compare to the finish time of any node in $B$? 

3. What happens to the strongly connected componenets if we reverse the direction of edges?

## Answers

1. This graph is also **acyclic**. Otherwise we would have had to merge the supernodes belonging to the cycle into a larger scc.

2. A > B

3. SCC nodes stay the same, but you could no longer get out of A. And if you had traversed A and started a new round in B, you couldn’t escape from that, leaving only C

# Kosaraju's algorithm:
Perform two rounds of DFS.
- The first one sorts out the nodes based on descending order of finishing time. Let's call the result L.
- The second one performs DFS on nodes from L in order

## Explanation:

If there’s an edge from A to B, A will have a later (final) finish time than B. If we choose starting points for our (second) traversal based on
decreasing finish times, this means that we’ll visit A before B. Now, if we reverse all the edges, we can still explore all of
A, but we can’t move on to B, and this lets us explore a single SCC at a time.

In [29]:
def tr(G): # Transpose (rev. edges of) G
    GT = {}
    for u in G: GT[u] = set() # Get all the nodes in there
    for u in G:
        for v in G[u]:
            GT[v].add(u) # Add all reverse edges
    return GT


def scc(G):
    GT = tr(G) # Get the transposed graph
    sccs, seen = [], set()
    for u in dfs_topsort(G): # DFS starting points
        if u in seen: continue # Ignore covered nodes
        C = walk(GT, u, seen) # Don't go "backward" (seen)
        seen.update(C) # We've now seen C
        sccs.append(C) # Another SCC found
    return sccs

## [Breadth First Traversal](https://www.hackerearth.com/practice/algorithms/graphs/breadth-first-search/visualize/)


In [25]:
# code for breadth first traversal
def bfs(G, s):
    P, Q = {s: None}, deque([s]) # Parents and FIFO queue
    while Q:
        u = Q.popleft() # Constant-time for deque
        for v in G[u]:
            if v in P: continue # Already has parent
            P[v] = u # Reached from u: u is parent
            Q.append(v)
    return P

## Question:
    - Why are we using a deque?

# BFS example: Dijkstra's Algorithm
##### [src](https://medium.com/basecs/finding-the-shortest-path-with-a-little-help-from-dijkstra-613149fbdc8e)

### Compute the shortest distance between node $a$ and node $e$
![](dijkstra.png)

## [Minimum Spanning Tree](https://www.hackerearth.com/practice/algorithms/graphs/minimum-spanning-tree/tutorial/)

- What is a tree?
- What is the weight of a tree?
- What is minimum spanning tree?


## This was an overview of the 5th chapter of [this book](https://www.amazon.com/Python-Algorithms-Mastering-Basic-Language-ebook/dp/B00LPDVBLS)